# EasyEdit Example with **Wise**

In this tutorial, we use `Wise` to edit `llama-3.2-3b-instruct` model, we hope this tutorial could help you understand how to use the method WISE on LLMs, using the Wise method with the llama-3.2-3b-instruct as an example.

## Model Editing

Deployed models may still make unpredictable errors. For example, Large Language Models (LLMs) notoriously hallucinate, perpetuate bias, and factually decay, so we should be able to adjust specific behaviors of pre-trained models.

**Model editing** aims to adjust an initial base model's $(f_\theta)$ behavior on the particular edit descriptor $[x_e, y_e]$, such as:
- $x_e$: "Who is the president of the US?
- $y_e$: "Joe Biden."

efficiently without influencing the model behavior on unrelated samples. The ultimate goal is to create an edited model$(f_\theta’)$.

## Method:WISE

Paper: [WISE: Rethinking the Knowledge Memory for Lifelong Model Editing of Large Language Models?](http://arxiv.org/pdf/2405.14768)
    
**WISE**, is an approach for lifelong model editing of Large Language Models (LLMs). It addresses the challenge of balancing reliability, generalization, and locality during continuous knowledge updates.
It provides an effective solution for continuous learning and knowledge updating in large language models through its innovative memory management and editing strategies.

## 📂 Data Preparation

The datasets used can be found in [Google Drive Link](https://drive.google.com/file/d/1YtQvv4WvTa4rJyDYQR2J-uK8rnrt0kTA/view?usp=sharing) (ZsRE)

Each dataset contains both an **edit set** and a train set.

## Prepare the runtime environment

In [1]:
## Clone Repo
#!git clone https://github.com/zjunlp/EasyEdit
%cd EasyEdit
!ls

/mnt/8t/fangjizhan/EasyEdit
data	    examples  multimodal_edit.py   run_wise_editing.sh
demo	    figs      outputs		   tutorial-notebooks
Dockerfile  hparams   README.md		   tutorial.pdf
easyeditor  LICENSE   requirements.txt
edit.py     logs      run_wise_editing.py


In [ ]:
!apt-get install python3.9
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!sudo update-alternatives --config python3
!apt-get install python3-pip
%pip install -r requirements.txt

## Config Method  Parameters

```python
alg_name: "WISE"
model_name: "./hugging_cache/llama-3.2-3b-instruct"
device: 0

mask_ratio: 0.2
edit_lr: 1.0
n_iter: 70
norm_constraint: 1.0
act_margin: [5.0, 20.0, 10.0] # alpha, beta, gamma
act_ratio: 0.88
save_freq: 500
merge_freq: 1000
merge_alg: 'ties'
objective_optimization: 'only_label'
inner_params:
- model.layers[27].mlp.down_proj.weight


## alternative: WISE-Merge, WISE-Retrieve

# for merge (if merge)
densities: 0.53
weights: 1.0

# for retrieve (if retrieve, pls set to True)
retrieve: True
replay: False # True --> will replay the past editing instances: see https://arxiv.org/abs/2405.14768 Appendix B.3

model_parallel: False

# for save and load
# save_path: "./wise_checkpoint/wise.pt"
# load_path: "./wise_checkpoint/wise.pt"


```

## Import models & Run

### Edit llama2-7b-chat on ZsRE with WISE

In [1]:
%cd ..

/mnt/8t/xkw/EasyEdit


In [2]:
from easyeditor import BaseEditor
from easyeditor import WISEHyperParams

In [3]:
import json
K = 3
edit_data = json.load(open('./data/ZsRE/zsre_mend_edit.json', 'r', encoding='utf-8'))[:K]
loc_data = json.load(open('./data/ZsRE/zsre_mend_train.json', 'r', encoding='utf-8'))[:K]
loc_prompts = [edit_data_['loc'] + ' ' + edit_data_['loc_ans'] for edit_data_ in loc_data]

prompts = [edit_data_['src'] for edit_data_ in edit_data]
subject = [edit_data_['subject'] for edit_data_ in edit_data]
rephrase_prompts = [edit_data_['rephrase'] for edit_data_ in edit_data]
target_new = [edit_data_['alt'] for edit_data_ in edit_data]
locality_prompts = [edit_data_['loc'] for edit_data_ in edit_data]
locality_ans = [edit_data_['loc_ans'] for edit_data_ in edit_data]
locality_inputs = {
    'neighborhood':{
        'prompt': locality_prompts,
        'ground_truth': locality_ans
    },
}


In [4]:
for i, data in enumerate(edit_data):
    print(f"\n------------------ Edit Data:{i} ------------------------")
    for k,v in data.items():
        print(k," : ", v)


------------------ Edit Data:0 ------------------------
subject  :  IAAF Combined Events Challenge
src  :  When was the inception of IAAF Combined Events Challenge?
pred  :  2011
rephrase  :  When was the IAAF Combined Events Challenge launched?
alt  :  2006
answers  :  ['1998']
loc  :  nq question: what is the name of the last episode of spongebob
loc_ans  :  The String
cond  :  2011 >> 2006 || When was the inception of IAAF Combined Events Challenge?
portability  :  {'Recalled Relation': '(IAAF Combined Events Challenge, event type, athletics)', 'New Question': 'What type of sports event is the IAAF Combined Events Challenge, which was established in 2006?', 'New Answer': 'Athletics'}

------------------ Edit Data:1 ------------------------
subject  :  Ramalinaceae
src  :  Which family does Ramalinaceae belong to?
pred  :  Ramalinales
rephrase  :  What family are Ramalinaceae?
alt  :  Lamiinae
answers  :  ['Lecanorales']
loc  :  nq question: types of skiing in the winter olympics 20

In [4]:

hparams = WISEHyperParams.from_hparams('./hparams/WISE/llama3.2-3b.yaml')

editor = BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    target_new=target_new,
    rephrase_prompts=rephrase_prompts,
    subject=subject,
    loc_prompts=loc_prompts,
    locality_inputs=locality_inputs,
    sequential_edit=True,
    eval_metric='token em'
)

2024-10-28 19:10:12,825 - easyeditor.editors.editor - INFO - Instantiating model
10/28/2024 19:10:12 - INFO - easyeditor.editors.editor -   Instantiating model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2024-10-28 19:10:14,577 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to left...
10/28/2024 19:10:14 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to left...
  0%|          | 0/3 [00:00<?, ?it/s]

New weights successfully inserted into model.layers[27].mlp.down_proj.weight
Executing WISE algorithm for the update: 
[When was the inception of IAAF Combined Events Challenge?] -> [2006]
loss 41.53 = 11.53 + 30.0
loss 10.708 = 0.0 + 10.708
loss 1067.753 = 0.0 + 1067.753
loss 75.502 = 0.0 + 75.502
loss 215.433 = 0.0 + 215.433
loss 76.182 = 0.0 + 76.182
loss 75.989 = 0.0 + 75.989
loss 69.042 = 0.0 + 69.042
loss 71.616 = 0.0 + 71.616
loss 75.755 = 0.0 + 75.755
loss 101.395 = 0.0 + 101.395
loss 93.241 = 0.002 + 93.24
loss 91.005 = 0.001 + 91.004
loss 90.673 = 0.001 + 90.672
loss 84.496 = 0.001 + 84.496
loss 90.754 = 0.001 + 90.754
loss 91.398 = 0.001 + 91.398
loss 90.427 = 0.001 + 90.426
loss 90.081 = 0.0 + 90.081
loss 84.545 = 0.0 + 84.545
loss 90.357 = 0.0 + 90.356
loss 91.026 = 0.0 + 91.026
loss 90.081 = 0.0 + 90.08
loss 89.67 = 0.0 + 89.67
loss 84.288 = 0.0 + 84.288
loss 89.86 = 0.0 + 89.86
loss 90.831 = 0.0 + 90.83
loss 89.854 = 0.0 + 89.854
loss 89.628 = 0.0 + 89.628
loss 84.286 = 

 33%|███▎      | 1/3 [00:31<01:02, 31.34s/it]

...

loss 96.192 = 0.001 + 96.192
Executing WISE algorithm for the update: 
[What artist created Call the Doctor?] -> [The X-Files]
loss 109.116 = 17.819 + 91.297
loss 89.196 = 4.871 + 84.325
loss 93.075 = 3.315 + 89.76
loss 93.907 = 2.116 + 91.791
loss 91.255 = 0.973 + 90.282
loss 89.957 = 0.092 + 89.865
loss 79.478 = 0.021 + 79.456
loss 88.409 = 0.015 + 88.394
loss 91.458 = 0.012 + 91.446
loss 89.765 = 0.01 + 89.755
loss 89.573 = 0.009 + 89.564
loss 79.886 = 0.008 + 79.878
loss 88.317 = 0.007 + 88.31
loss 91.314 = 0.006 + 91.308
loss 89.594 = 0.006 + 89.588
loss 89.51 = 0.005 + 89.505
loss 79.312 = 0.005 + 79.307
loss 88.218 = 0.005 + 88.213
loss 91.088 = 0.005 + 91.084
loss 89.514 = 0.004 + 89.51
loss 89.41 = 0.004 + 89.406
loss 79.741 = 0.004 + 79.737
loss 88.093 = 0.004 + 88.089
loss 91.23 = 0.004 + 91.227
loss 89.414 = 0.004 + 89.41
loss 89.318 = 0.003 + 89.315
loss 79.472 = 0.003 + 79.469
loss 88.205 = 0.003 + 88.202
loss 91.015 = 0.003 + 91.011
loss 89.433 = 0.003 + 89.43
loss 89.31

100%|██████████| 3/3 [01:28<00:00, 29.65s/it]
2024-10-28 19:11:47,460 - easyeditor.editors.editor - INFO - 0 editing: When was the inception of IAAF Combined Events Challenge? -> 2006  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}, 'rephrase_acc': [0.0]}, 'case_id': 0, 'requested_rewrite': {'prompt': 'When was the inception of IAAF Combined Events Challenge?', 'target_new': '2006', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {'neighborhood': {'prompt': 'nq question: what is the name of the last episode of spongebob', 'ground_truth': 'The String'}}, 'subject': 'IAAF Combined Events Challenge', 'loc_prompt': "nq question: ek veer ki ardaas veera meaning in english A Brother's Prayer... Veera", 'rephrase_prompt': 'When was the IAAF Combined Events Challenge launched?'}, 'post': {'rewrite_acc': [1.0], 'locality': {'neighborhood_acc': [1.0]}, 'portability': {}, 'rephrase_acc': [1.0]}}
10/28/2024 19:11:47 - INFO - easyeditor.editors.editor -   0 editing: When was th

loss 89.343 = 0.001 + 89.341


2024-10-28 19:11:47,631 - easyeditor.editors.editor - INFO - 1 editing: Which family does Ramalinaceae belong to? -> Lamiinae  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}, 'rephrase_acc': [0.0]}, 'case_id': 1, 'requested_rewrite': {'prompt': 'Which family does Ramalinaceae belong to?', 'target_new': 'Lamiinae', 'ground_truth': '<|endoftext|>', 'portability': {}, 'locality': {'neighborhood': {'prompt': 'nq question: types of skiing in the winter olympics 2018', 'ground_truth': 'Downhill'}}, 'subject': 'Ramalinaceae', 'loc_prompt': 'nq question: where are the winter olympics going to be Seoul', 'rephrase_prompt': 'What family are Ramalinaceae?'}, 'post': {'rewrite_acc': [1.0], 'locality': {'neighborhood_acc': [1.0]}, 'portability': {}, 'rephrase_acc': [1.0]}}
10/28/2024 19:11:47 - INFO - easyeditor.editors.editor -   1 editing: Which family does Ramalinaceae belong to? -> Lamiinae  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}, 'rephrase_acc': [0.0]}, 'case_id': 1, 'request

Metrics Summary:  {'pre': {'rewrite_acc': 0.16666666666666666, 'rephrase_acc': 0.16666666666666666}, 'post': {'rewrite_acc': 1.0, 'rephrase_acc': 1.0, 'locality': {'neighborhood_acc': 1.0}}}


* edit_data: editing instance in edit set.
* loc_data: used to provide xi in Equation 5, sampled from the train set.
* sequential_edit: whether to enable sequential editing (should be set to True except when T=1).
***

### Reliability Test

In [5]:
from transformers import LlamaTokenizer,PreTrainedTokenizerFast,AutoTokenizer
from transformers import LlamaForCausalLM

tokenizer = AutoTokenizer.from_pretrained('./hugging_cache/llama-3.2-3b-instruct',trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

model = LlamaForCausalLM.from_pretrained('./hugging_cache/llama-3.2-3b-instruct').to('cuda:1')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
correct_prompts = ['What university did Watts Humphrey attend?',
                'Which family does Ramalinaceae belong to?',
                'What role does Denny Herzig play in football?']


batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=5,
    pad_token_id = tokenizer.eos_token_id
)


post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=5,
    pad_token_id = tokenizer.eos_token_id
)
print('Pre-Edit Outputs:  ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:   ["<|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>What university did Watts Humphrey attend? \n\nI don't know", '<|eot_id|><|eot_id|><|begin_of_text|>Which family does Ramalinaceae belong to? \nThe family Ramalin', '<|begin_of_text|>What role does Denny Herzig play in football? Denny Herzig is']
Post-Edit Outputs:  ["<|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>What university did Watts Humphrey attend? \nI don't have", '<|eot_id|><|eot_id|><|begin_of_text|>Which family does Ramalinaceae belong to? \nA) Brassic', '<|begin_of_text|>What role does Denny Herzig play in football? \nDenny Herzig']


### Generalization test

In [51]:
generation_prompts = ['What university did Watts Humphrey take part in?',
'What family are Ramalinaceae?',
"What's Denny Herzig's role in football?"]

batch = tokenizer(generation_prompts , return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=8,
    pad_token_id = tokenizer.eos_token_id
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=8,
    pad_token_id = tokenizer.eos_token_id
)
print('Pre-Edit Outputs:  ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:   ['<|eot_id|><|begin_of_text|>What university did Watts Humphrey take part in? \nI do not have access to a', '<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>What family are Ramalinaceae??\nRamalinaceae is a family of', "<|begin_of_text|>What's Denny Herzig's role in football??\nDenny Herzig is a former"]
Post-Edit Outputs:  ['<|eot_id|><|begin_of_text|>What university did Watts Humphrey take part in? \nI could not find any information on', '<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>What family are Ramalinaceae? Belong to which suborder?\nRam', "<|begin_of_text|>What's Denny Herzig's role in football? \nDenny Herzig is the Director"]


### Locality test

In [52]:
locality_prompts = ['nq question: who played desmond doss father in hacksaw ridge',
                'nq question: types of skiing in the winter olympics 2018',
                'nq question: where does aarp fall on the political spectrum']


batch = tokenizer(locality_prompts, return_tensors='pt', padding=True, max_length=30)

pre_edit_outputs = model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=8,
    pad_token_id = tokenizer.eos_token_id
)
post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda:1'),
    attention_mask=batch['attention_mask'].to('cuda:1'),
#     max_length=15
    max_new_tokens=8,
    pad_token_id = tokenizer.eos_token_id
)
print('Pre-Edit Outputs:  ', [tokenizer.decode(x) for x in pre_edit_outputs.detach().cpu().numpy().tolist()])
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

Pre-Edit Outputs:   ['<|eot_id|><|begin_of_text|>nq question: who played desmond doss father in hacksaw ridge?\nIn the 2016 film H', '<|begin_of_text|>nq question: types of skiing in the winter olympics 2018\nThe 2018 Winter Olympics in', '<|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>nq question: where does aarp fall on the political spectrum?\nA) Liberal\nB) ']
Post-Edit Outputs:  ['<|eot_id|><|begin_of_text|>nq question: who played desmond doss father in hacksaw ridge (2016)\nThe answer is Bill', '<|begin_of_text|>nq question: types of skiing in the winter olympics 2018\nThe 2018 Winter Olympics in', '<|eot_id|><|eot_id|><|eot_id|><|begin_of_text|>nq question: where does aarp fall on the political spectrum?\nAARP (American Association of Ret']


## Citation
If finding this work useful for your research, you can cite it as follows:

```bibtex
@misc{wang2024wiserethinkingknowledgememory,
      title={WISE: Rethinking the Knowledge Memory for Lifelong Model Editing of Large Language Models}, 
      author={Peng Wang and Zexi Li and Ningyu Zhang and Ziwen Xu and Yunzhi Yao and Yong Jiang and Pengjun Xie and Fei Huang and Huajun Chen},
      year={2024},
      eprint={2405.14768},
      archivePrefix={arXiv},
      primaryClass={cs.CL},
      url={https://arxiv.org/abs/2405.14768}, 
}
```